In [50]:
import re
import patsy
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from sklearn.preprocessing import scale
import warnings
import pickle

In [51]:
def makeFormula(target, numeric=None, categoric=None, noInter = False, scale = False):
    formula = ""
    formula += target
    formula += "~"
    
    if(numeric != None):
        if scale == True:
            numeric = list(map(lambda c :  "scale(" + c + ")", numeric))
        formula_numeric = "+".join(numeric)
        formula += formula_numeric 
    
    if(numeric != None and categoric != None):
        formula += "+"
    
    if(categoric != None):
        categoric = list(map(lambda c :  "C(" + c + ")", categoric))
        formula_categoric = "+".join(categoric)
        formula += formula_categoric
    
    if(noInter == True):
        formula +="+0"
    
    return formula

In [52]:
df_train = pd.read_csv("C:/Users/user/Desktop/newww/house-prj/train_no_missing.csv")
df_test = pd.read_csv("C:/Users/user/Desktop/newww/house-prj/test_no_missing.csv")

# Preprocessing

In [53]:
nuTocaList = ["MSSubClass", "MoSold", "YrSold", "OverallQual", "OverallCond"]
df_train[nuTocaList] = df_train[nuTocaList].astype("str")
df_test[nuTocaList] = df_test[nuTocaList].astype("str")

df_all = pd.concat([df_train, df_test])
df_all.reset_index(drop = True, inplace = True)

df_all = pd.get_dummies(df_all)
df_train_dummy = df_all.iloc[:1460]
df_test_dummy = df_all.iloc[1460:]

df_train_dummy.reset_index(drop=True, inplace =True)
df_test_dummy.reset_index(drop=True, inplace =True)

df_test_dummy.drop("SalePriceLog", axis = 1, inplace = True)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [54]:
df_train.shape, df_test.shape, df_train_dummy.shape, df_test_dummy.shape

((1460, 81), (1459, 80), (1460, 351), (1459, 350))

# Split into numerical and categorical data 

In [55]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_list = list(df_test.select_dtypes(include=numerics).columns)

print(len(numeric_list))

32


In [56]:
df_scale_numeric = pd.DataFrame()
for c in numeric_list:
    if c == "Id":
        continue
    df_scale_numeric[c+"Scale"] = scale(df_train[c])
    
df_scale_numeric.drop("TotalBsmtSFScale", axis = 1, inplace = True)
df_scale_numeric.drop("GrLivAreaScale", axis= 1, inplace=True)

df_scale_numeric["SalePriceLog"] = df_train["SalePriceLog"]

scale_list = list(df_scale_numeric.columns)
scale_list.remove('SalePriceLog')

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [57]:
categoric_list = list(df_train.select_dtypes(include=['object']).columns)
categoric_dummy_list = list(df_train_dummy.filter(regex="_").columns)

df_scale_cate = pd.concat([df_scale_numeric, df_train[categoric_list]], axis=1)
df_scale_cate_dummy = pd.concat([df_scale_numeric, df_train_dummy[categoric_dummy_list]], axis=1)

df_scale_cate.shape, df_scale_cate_dummy.shape

((1460, 78), (1460, 348))

In [58]:
column_ls = scale_list + categoric_dummy_list

# Modeling

- scale numerical data + categorical dummy data
- test score : 0.1498

In [59]:
model = sm.OLS.from_formula(makeFormula("SalePriceLog", column_ls, None, True),df_scale_cate_dummy)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           SalePriceLog   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.935
Method:                 Least Squares   F-statistic:                     73.74
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:05:47   Log-Likelihood:                 1423.3
No. Observations:                1460   AIC:                            -2269.
Df Residuals:                    1171   BIC:                            -740.8
Df Model:                         288                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
FirstFlrSFScale           0.0893      0.010      9.140      0.000       0.070       0.108
SecondFlrSFScale          0.0986      0.012      8.521      0.000       0.076       0.121
ThreeSsnPorchScale        0.0047      0.003      1.572      0.116      -0.001       0.011
BedroomAbvGrScale         0.0019      0.005      0.366      0.715      -0.008       0.012
BsmtFinSF1Scale           0.0617      0.011      5.732      0.000       0.041       0.083
BsmtFinSF2Scale           0.0205      0.007      3.139      0.002       0.008       0.033
BsmtFullBathScale         0.0127      0.005      2.702      0.007       0.003       0.022
BsmtHalfBathScale         0.0030      0.003      0.912      0.362      -0.003       0.010
BsmtUnfSFScale            0.0363      0.009      3.835      0.000       0.018       0.055
EnclosedPorchScale        0.0073      0.004      2.062      0.039       0.000       0.014
FireplacesScale           0.0097      0.008      1.279      0.201      -0.005       0.024
FullBathScale             0.0161      0.006      2.862      0.004       0.005       0.027
GarageAreaScale           0.0243      0.008      3.112      0.002       0.009       0.040
GarageCarsScale           0.0146      0.008      1.857      0.064      -0.001       0.030
GarageYrBltScale         -0.0011      0.007     -0.159      0.874      -0.015       0.013
HalfBathScale             0.0149      0.005      3.073      0.002       0.005       0.024
KitchenAbvGrScale        -0.0134      0.006     -2.138      0.033      -0.026      -0.001
LotAreaScale              0.0275      0.005      5.460      0.000       0.018       0.037
LotFrontageScale          0.0092      0.005      1.969      0.049    3.46e-05       0.018
LowQualFinSFScale         0.0075      0.004      1.726      0.085      -0.001       0.016
MasVnrAreaScale           0.0035      0.005      0.735      0.463      -0.006       0.013
MiscValScale              0.0175      0.017      1.011      0.312      -0.016       0.051
OpenPorchSFScale          0.0029      0.004      0.813      0.416      -0.004       0.010
PoolAreaScale             0.0684      0.042      1.616      0.106      -0.015       0.151
ScreenPorchScale          0.0158      0.003      4.965      0.000       0.010       0.022
TotRmsAbvGrdScale         0.0026      0.007      0.358      0.720      -0.011       0.017
WoodDeckSFScale           0.0118      0.003      3.515      0.000       0.005       0.018
YearBuiltScale            0.0527      0.012      4.353      0.000       0.029       0.076
YearRemodAddScale         0.0179      0.005      3.364      0.001       0.007       0.028
Alley_Grvl                0.3499      0.022     16.136      0.000       0.307       0.392
Alley_None                0.3442      0.019     18.168      0.000       0.307       0.381
Alley_Pave             

## trial 1
- delete the variables that have over 0.01 p-value
- test score: 0.1449

In [60]:
over_pvalues = list(result.pvalues[result.pvalues > 0.01].index)
for c in over_pvalues:
    column_ls.remove(c)
    
model1 = sm.OLS.from_formula(makeFormula("SalePriceLog", column_ls, None, True),df_scale_cate_dummy)
result1 = model1.fit()
result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           SalePriceLog   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                     98.49
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:05:48   Log-Likelihood:                 1333.4
No. Observations:                1460   AIC:                            -2259.
Df Residuals:                    1256   BIC:                            -1180.
Df Model:                         203                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
FirstFlrSFScale           0.0968      0.008     11.487      0.000       0.080       0.113
SecondFlrSFScale          0.1043      0.009     11.504      0.000       0.087       0.122
BsmtFinSF1Scale           0.0615      0.010      5.928      0.000       0.041       0.082
BsmtFinSF2Scale           0.0242      0.006      3.766      0.000       0.012       0.037
BsmtFullBathScale         0.0104      0.004      2.376      0.018       0.002       0.019
BsmtUnfSFScale            0.0357      0.009      3.953      0.000       0.018       0.053
FullBathScale             0.0177      0.005      3.347      0.001       0.007       0.028
GarageAreaScale           0.0349      0.005      6.815      0.000       0.025       0.045
HalfBathScale             0.0145      0.005      3.083      0.002       0.005       0.024
LotAreaScale              0.0290      0.005      6.223      0.000       0.020       0.038
ScreenPorchScale          0.0134      0.003      4.370      0.000       0.007       0.019
WoodDeckSFScale           0.0098      0.003      2.966      0.003       0.003       0.016
YearBuiltScale            0.0559      0.010      5.733      0.000       0.037       0.075
YearRemodAddScale         0.0191      0.005      3.666      0.000       0.009       0.029
Alley_Grvl                0.5157      0.020     25.851      0.000       0.477       0.555
Alley_None                0.4979      0.017     29.078      0.000       0.464       0.531
Alley_Pave                0.5233      0.021     25.109      0.000       0.482       0.564
BldgType_1Fam            -0.0126      0.026     -0.490      0.625      -0.063       0.038
BldgType_Duplex          -0.0313      0.016     -1.965      0.050      -0.062   -4.31e-05
BldgType_Twnhs           -0.1155      0.033     -3.532      0.000      -0.180      -0.051
BldgType_TwnhsE          -0.0653      0.030     -2.182      0.029      -0.124      -0.007
BsmtCond_Fa               0.2172      0.027      7.954      0.000       0.164       0.271
BsmtCond_Gd               0.2487      0.027      9.295      0.000       0.196       0.301
BsmtCond_None             0.2584      0.011     24.211      0.000       0.237       0.279
BsmtCond_Po               0.5594      0.076      7.358      0.000       0.410       0.708
BsmtCond_TA               0.2531      0.024     10.341      0.000       0.205       0.301
BsmtExposure_Av           0.3155      0.011     27.594      0.000       0.293       0.338
BsmtExposure_Gd           0.3517      0.013     26.725      0.000       0.326       0.378
BsmtExposure_Mn           0.3068      0.013     23.381      0.000       0.281       0.333
BsmtExposure_No           0.3043      0.011     28.581      0.000       0.283       0.325
BsmtExposure_None         0.2584      0.011     24.211      0.000       0.237       0.279
BsmtFinType1_ALQ       

## trial2
- test score: 0.1441

In [61]:
over_pvalues = list(result1.pvalues[result1.pvalues > 0.01].index)
for c in over_pvalues:
    column_ls.remove(c)
    
model2 = sm.OLS.from_formula(makeFormula("SalePriceLog", column_ls, None, True),df_scale_cate_dummy)
result2 = model2.fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           SalePriceLog   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     127.4
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:05:48   Log-Likelihood:                 1232.8
No. Observations:                1460   AIC:                            -2180.
Df Residuals:                    1317   BIC:                            -1424.
Df Model:                         142                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
FirstFlrSFScale           0.0984      0.008     12.322      0.000       0.083       0.114
SecondFlrSFScale          0.1099      0.006     18.836      0.000       0.098       0.121
BsmtFinSF1Scale           0.0777      0.010      8.067      0.000       0.059       0.097
BsmtFinSF2Scale           0.0294      0.006      4.580      0.000       0.017       0.042
BsmtUnfSFScale            0.0442      0.009      5.103      0.000       0.027       0.061
FullBathScale             0.0103      0.005      1.980      0.048    9.57e-05       0.021
GarageAreaScale           0.0339      0.005      6.864      0.000       0.024       0.044
HalfBathScale             0.0098      0.005      2.161      0.031       0.001       0.019
LotAreaScale              0.0300      0.005      6.500      0.000       0.021       0.039
ScreenPorchScale          0.0134      0.003      4.286      0.000       0.007       0.019
WoodDeckSFScale           0.0112      0.003      3.328      0.001       0.005       0.018
YearBuiltScale            0.0499      0.008      5.953      0.000       0.033       0.066
YearRemodAddScale         0.0203      0.005      3.885      0.000       0.010       0.031
Alley_Grvl                0.5510      0.018     30.232      0.000       0.515       0.587
Alley_None                0.5405      0.015     36.826      0.000       0.512       0.569
Alley_Pave                0.5616      0.019     29.774      0.000       0.525       0.599
BldgType_Twnhs           -0.0845      0.020     -4.231      0.000      -0.124      -0.045
BsmtCond_Fa               0.1908      0.049      3.887      0.000       0.095       0.287
BsmtCond_Gd               0.2170      0.049      4.426      0.000       0.121       0.313
BsmtCond_None             0.2696      0.013     21.028      0.000       0.244       0.295
BsmtCond_Po               0.7525      0.129      5.854      0.000       0.500       1.005
BsmtCond_TA               0.2231      0.048      4.676      0.000       0.130       0.317
BsmtExposure_Av           0.3414      0.009     36.263      0.000       0.323       0.360
BsmtExposure_Gd           0.3733      0.012     31.935      0.000       0.350       0.396
BsmtExposure_Mn           0.3372      0.011     30.554      0.000       0.316       0.359
BsmtExposure_No           0.3315      0.008     40.238      0.000       0.315       0.348
BsmtExposure_None         0.2696      0.013     21.028      0.000       0.244       0.295
BsmtFinType1_ALQ          0.2379      0.009     26.172      0.000       0.220       0.256
BsmtFinType1_BLQ          0.2314      0.010     23.085      0.000       0.212       0.251
BsmtFinType1_GLQ          0.2567      0.009     27.785      0.000       0.239       0.275
BsmtFinType1_LwQ          0.2091      0.013     15.891      0.000       0.183       0.235
BsmtFinType1_None      

## trial3
- test score: 0.1398

In [62]:
over_pvalues = list(result2.pvalues[result2.pvalues > 0.01].index)
for c in over_pvalues:
    column_ls.remove(c)
    
model3 = sm.OLS.from_formula(makeFormula("SalePriceLog", column_ls, None, True),df_scale_cate_dummy)
result3 = model3.fit()
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           SalePriceLog   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     132.9
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:05:49   Log-Likelihood:                 1217.6
No. Observations:                1460   AIC:                            -2165.
Df Residuals:                    1325   BIC:                            -1452.
Df Model:                         134                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
FirstFlrSFScale           0.1045      0.008     13.540      0.000       0.089       0.120
SecondFlrSFScale          0.1202      0.004     29.306      0.000       0.112       0.128
BsmtFinSF1Scale           0.0743      0.010      7.698      0.000       0.055       0.093
BsmtFinSF2Scale           0.0289      0.006      4.491      0.000       0.016       0.042
BsmtUnfSFScale            0.0413      0.009      4.755      0.000       0.024       0.058
GarageAreaScale           0.0342      0.005      6.904      0.000       0.024       0.044
LotAreaScale              0.0278      0.005      6.058      0.000       0.019       0.037
ScreenPorchScale          0.0141      0.003      4.504      0.000       0.008       0.020
WoodDeckSFScale           0.0108      0.003      3.203      0.001       0.004       0.017
YearBuiltScale            0.0512      0.008      6.297      0.000       0.035       0.067
YearRemodAddScale         0.0208      0.005      3.971      0.000       0.011       0.031
Alley_Grvl                0.5779      0.015     38.209      0.000       0.548       0.608
Alley_None                0.5667      0.011     50.399      0.000       0.545       0.589
Alley_Pave                0.5861      0.016     35.775      0.000       0.554       0.618
BldgType_Twnhs           -0.0791      0.020     -3.959      0.000      -0.118      -0.040
BsmtCond_Fa               0.2803      0.030      9.281      0.000       0.221       0.340
BsmtCond_Gd               0.3083      0.030     10.291      0.000       0.250       0.367
BsmtCond_None             0.2939      0.008     38.550      0.000       0.279       0.309
BsmtCond_Po               0.5325      0.082      6.469      0.000       0.371       0.694
BsmtCond_TA               0.3157      0.028     11.439      0.000       0.262       0.370
BsmtExposure_Av           0.3528      0.008     43.277      0.000       0.337       0.369
BsmtExposure_Gd           0.3872      0.010     37.134      0.000       0.367       0.408
BsmtExposure_Mn           0.3513      0.010     34.998      0.000       0.332       0.371
BsmtExposure_No           0.3456      0.007     49.628      0.000       0.332       0.359
BsmtExposure_None         0.2939      0.008     38.550      0.000       0.279       0.309
BsmtFinType1_ALQ          0.2466      0.009     28.932      0.000       0.230       0.263
BsmtFinType1_BLQ          0.2397      0.009     25.478      0.000       0.221       0.258
BsmtFinType1_GLQ          0.2633      0.009     30.379      0.000       0.246       0.280
BsmtFinType1_LwQ          0.2167      0.013     16.808      0.000       0.191       0.242
BsmtFinType1_None         0.2939      0.008     38.550      0.000       0.279       0.309
BsmtFinType1_Rec          0.2374      0.010     23.709      0.000       0.218       0.257
BsmtFinType1_Unf       

## trial4
- test score: 0.1395

In [63]:
over_pvalues = list(result3.pvalues[result3.pvalues > 0.01].index)
for c in over_pvalues:
    column_ls.remove(c)
    
model4 = sm.OLS.from_formula(makeFormula("SalePriceLog", column_ls, None, True),df_scale_cate_dummy)
result4 = model4.fit()
result4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           SalePriceLog   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     136.7
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:05:49   Log-Likelihood:                 1208.7
No. Observations:                1460   AIC:                            -2157.
Df Residuals:                    1330   BIC:                            -1470.
Df Model:                         129                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
FirstFlrSFScale           0.1032      0.008     13.356      0.000       0.088       0.118
SecondFlrSFScale          0.1214      0.004     29.673      0.000       0.113       0.129
BsmtFinSF1Scale           0.0749      0.010      7.737      0.000       0.056       0.094
BsmtFinSF2Scale           0.0296      0.006      4.584      0.000       0.017       0.042
BsmtUnfSFScale            0.0423      0.009      4.868      0.000       0.025       0.059
GarageAreaScale           0.0334      0.005      6.727      0.000       0.024       0.043
LotAreaScale              0.0275      0.005      5.962      0.000       0.018       0.037
ScreenPorchScale          0.0134      0.003      4.286      0.000       0.007       0.020
WoodDeckSFScale           0.0104      0.003      3.082      0.002       0.004       0.017
YearBuiltScale            0.0444      0.008      5.619      0.000       0.029       0.060
YearRemodAddScale         0.0204      0.005      3.922      0.000       0.010       0.031
Alley_Grvl                0.5807      0.015     38.514      0.000       0.551       0.610
Alley_None                0.5730      0.011     54.211      0.000       0.552       0.594
Alley_Pave                0.5983      0.016     37.824      0.000       0.567       0.629
BldgType_Twnhs           -0.0813      0.020     -4.055      0.000      -0.121      -0.042
BsmtCond_Fa               0.3081      0.027     11.423      0.000       0.255       0.361
BsmtCond_Gd               0.3312      0.027     12.253      0.000       0.278       0.384
BsmtCond_None             0.3006      0.007     43.192      0.000       0.287       0.314
BsmtCond_Po               0.4752      0.076      6.290      0.000       0.327       0.623
BsmtCond_TA               0.3369      0.025     13.697      0.000       0.289       0.385
BsmtExposure_Av           0.3563      0.008     44.404      0.000       0.341       0.372
BsmtExposure_Gd           0.3916      0.010     38.029      0.000       0.371       0.412
BsmtExposure_Mn           0.3546      0.010     35.708      0.000       0.335       0.374
BsmtExposure_No           0.3488      0.007     51.256      0.000       0.335       0.362
BsmtExposure_None         0.3006      0.007     43.192      0.000       0.287       0.314
BsmtFinType1_ALQ          0.2478      0.008     29.278      0.000       0.231       0.264
BsmtFinType1_BLQ          0.2427      0.009     25.852      0.000       0.224       0.261
BsmtFinType1_GLQ          0.2669      0.009     31.024      0.000       0.250       0.284
BsmtFinType1_LwQ          0.2194      0.013     17.028      0.000       0.194       0.245
BsmtFinType1_None         0.3006      0.007     43.192      0.000       0.287       0.314
BsmtFinType1_Rec          0.2389      0.010     23.947      0.000       0.219       0.259
BsmtFinType1_Unf       

## trial5
- delete the category data that have over 0.01 F-value
- test score: 0.1353

In [64]:
df_one_anova_Fvalue = pd.DataFrame(columns = ["name", "Fvalue"])

for idx, c in enumerate(categoric_list):
    model = sm.OLS.from_formula("SalePriceLog ~ C({})".format(c), data=df_train)
    result = model.fit()
    df_tmp=sm.stats.anova_lm(result)
    df_one_anova_Fvalue.loc[idx] = {"name":c, "Fvalue":df_tmp['PR(>F)'][0]}   

df_one_anova_Fvalue.sort_values(by=["Fvalue"], ascending=False, inplace=True)
df_one_anova_Fvalue.reset_index(drop=True, inplace=True)
df_one_anova_Fvalue

C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,name,Fvalue
0,Utilities,6.296086e-01
1,YrSold,5.656386e-01
2,MoSold,4.496614e-01
3,LandSlope,3.388339e-01
4,Street,2.837965e-02
5,Condition2,1.382043e-02
6,PoolQC,7.695369e-03
7,MiscFeature,6.060004e-03
8,RoofMatl,4.504077e-04
9,Functional,4.250494e-06


In [65]:
over_one_Fvalue_list = list(df_one_anova_Fvalue[df_one_anova_Fvalue["Fvalue"] > 0.01]["name"])
over_one_Fvalue_dummy_list=[]
for c in over_one_Fvalue_list:
    reg = c+"_"
    tmp = list(df_test_dummy.filter(regex = reg))
    for a in tmp:
        over_one_Fvalue_dummy_list.append(a)

In [66]:
for c in over_one_Fvalue_dummy_list:
    if c in column_ls:
        column_ls.remove(c)

In [67]:
model5 = sm.OLS.from_formula(makeFormula("SalePriceLog", column_ls, None, True),df_scale_cate_dummy)
result5 = model5.fit()
result5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           SalePriceLog   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                     146.9
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:05:50   Log-Likelihood:                 1133.0
No. Observations:                1460   AIC:                            -2046.
Df Residuals:                    1350   BIC:                            -1464.
Df Model:                         109                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
FirstFlrSFScale           0.1036      0.008     12.960      0.000       0.088       0.119
SecondFlrSFScale          0.1213      0.004     28.764      0.000       0.113       0.130
BsmtFinSF1Scale           0.0686      0.010      6.866      0.000       0.049       0.088
BsmtFinSF2Scale           0.0282      0.007      4.214      0.000       0.015       0.041
BsmtUnfSFScale            0.0400      0.009      4.451      0.000       0.022       0.058
GarageAreaScale           0.0325      0.005      6.337      0.000       0.022       0.043
LotAreaScale              0.0168      0.004      4.255      0.000       0.009       0.025
ScreenPorchScale          0.0136      0.003      4.222      0.000       0.007       0.020
WoodDeckSFScale           0.0105      0.004      2.991      0.003       0.004       0.017
YearBuiltScale            0.0414      0.008      5.038      0.000       0.025       0.058
YearRemodAddScale         0.0192      0.005      3.558      0.000       0.009       0.030
Alley_Grvl                0.8004      0.016     50.402      0.000       0.769       0.832
Alley_None                0.7932      0.011     72.528      0.000       0.772       0.815
Alley_Pave                0.8208      0.016     49.753      0.000       0.788       0.853
BldgType_Twnhs           -0.0898      0.021     -4.313      0.000      -0.131      -0.049
BsmtCond_Fa               0.4614      0.028     16.745      0.000       0.407       0.515
BsmtCond_Gd               0.4889      0.027     17.788      0.000       0.435       0.543
BsmtCond_None             0.4205      0.007     60.328      0.000       0.407       0.434
BsmtCond_Po               0.5517      0.078      7.117      0.000       0.400       0.704
BsmtCond_TA               0.4920      0.025     19.680      0.000       0.443       0.541
BsmtExposure_Av           0.4893      0.008     58.011      0.000       0.473       0.506
BsmtExposure_Gd           0.5326      0.011     49.933      0.000       0.512       0.554
BsmtExposure_Mn           0.4902      0.010     47.226      0.000       0.470       0.511
BsmtExposure_No           0.4818      0.007     66.969      0.000       0.468       0.496
BsmtExposure_None         0.4205      0.007     60.328      0.000       0.407       0.434
BsmtFinType1_ALQ          0.3420      0.009     38.809      0.000       0.325       0.359
BsmtFinType1_BLQ          0.3347      0.010     34.262      0.000       0.316       0.354
BsmtFinType1_GLQ          0.3559      0.009     39.307      0.000       0.338       0.374
BsmtFinType1_LwQ          0.3070      0.013     22.829      0.000       0.281       0.333
BsmtFinType1_None         0.4205      0.007     60.328      0.000       0.407       0.434
BsmtFinType1_Rec          0.3317      0.010     32.007      0.000       0.311       0.352
BsmtFinType1_Unf       

## trial6
- delete the category class data that have over 0.01 F-value
- test score: 0.1418

In [73]:
df_one_anova_class_Fvalue = pd.DataFrame(columns = ["name", "Fvalue"])

for idx, c in enumerate(categoric_dummy_list):
    model = sm.OLS.from_formula("SalePriceLog ~ C({})".format(c), data=df_train_dummy)
    result = model.fit()
    df_tmp=sm.stats.anova_lm(result)
    df_one_anova_class_Fvalue.loc[idx] = {"name":c, "Fvalue":df_tmp['PR(>F)'][0]}   

df_one_anova_class_Fvalue.sort_values(by=["Fvalue"], ascending=False, inplace=True)
df_one_anova_class_Fvalue.reset_index(drop=True, inplace=True)
df_one_anova_class_Fvalue

C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,name,Fvalue
0,MSSubClass_80,9.654636e-01
1,MiscFeature_Gar2,9.505640e-01
2,BsmtFinType2_GLQ,9.286725e-01
3,MoSold_10,9.240021e-01
4,RoofMatl_ClyTile,9.180190e-01
5,SaleType_ConLI,9.070738e-01
6,LotConfig_Corner,8.790832e-01
7,MoSold_3,8.707371e-01
8,MoSold_6,8.571153e-01
9,LotConfig_FR2,8.521263e-01


In [74]:
over_one_FvalueClass_list = list(df_one_anova_class_Fvalue[df_one_anova_class_Fvalue["Fvalue"] > 0.01]["name"])
for c in over_one_FvalueClass_list:
    if c in column_ls:
        column_ls.remove(c)

In [75]:
model6 = sm.OLS.from_formula(makeFormula("SalePriceLog", column_ls, None, True),df_scale_cate_dummy)
result6 = model6.fit()
result6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           SalePriceLog   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                     134.5
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:20:35   Log-Likelihood:                 930.23
No. Observations:                1460   AIC:                            -1680.
Df Residuals:                    1370   BIC:                            -1205.
Df Model:                          89                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
FirstFlrSFScale           0.1150      0.009     13.073      0.000       0.098       0.132
SecondFlrSFScale          0.1089      0.005     23.189      0.000       0.100       0.118
BsmtFinSF1Scale           0.0040      0.010      0.386      0.700      -0.016       0.024
BsmtFinSF2Scale           0.0183      0.007      2.637      0.008       0.005       0.032
BsmtUnfSFScale            0.0013      0.010      0.133      0.894      -0.018       0.020
GarageAreaScale           0.0334      0.006      5.777      0.000       0.022       0.045
LotAreaScale              0.0203      0.004      4.748      0.000       0.012       0.029
ScreenPorchScale          0.0155      0.004      4.224      0.000       0.008       0.023
WoodDeckSFScale           0.0123      0.004      3.088      0.002       0.004       0.020
YearBuiltScale            0.0225      0.009      2.493      0.013       0.005       0.040
YearRemodAddScale         0.0295      0.006      5.127      0.000       0.018       0.041
Alley_Grvl               -0.0230      0.032     -0.723      0.470      -0.086       0.039
Alley_None               -0.0275      0.025     -1.116      0.264      -0.076       0.021
BldgType_Twnhs           -0.0993      0.024     -4.217      0.000      -0.145      -0.053
BsmtCond_Fa               1.0533      0.031     33.591      0.000       0.992       1.115
BsmtCond_Gd               1.1359      0.030     37.256      0.000       1.076       1.196
BsmtCond_None             0.5613      0.009     63.007      0.000       0.544       0.579
BsmtCond_Po               1.0262      0.089     11.486      0.000       0.851       1.201
BsmtCond_TA               1.1204      0.028     40.156      0.000       1.066       1.175
BsmtExposure_Av          -0.0024      0.016     -0.152      0.880      -0.034       0.029
BsmtExposure_Gd           0.0621      0.019      3.338      0.001       0.026       0.099
BsmtExposure_No          -0.0140      0.014     -1.027      0.304      -0.041       0.013
BsmtExposure_None         0.5613      0.009     63.007      0.000       0.544       0.579
BsmtFinType1_ALQ          0.7446      0.012     63.034      0.000       0.721       0.768
BsmtFinType1_BLQ          0.7259      0.013     57.503      0.000       0.701       0.751
BsmtFinType1_GLQ          0.7514      0.012     62.224      0.000       0.728       0.775
BsmtFinType1_LwQ          0.7052      0.016     43.145      0.000       0.673       0.737
BsmtFinType1_None         0.5613      0.009     63.007      0.000       0.544       0.579
BsmtFinType1_Rec          0.7284      0.013     55.789      0.000       0.703       0.754
BsmtFinType1_Unf          0.6803      0.012     55.878      0.000       0.656       0.704
BsmtFinType2_None         0.5613      0.009     63.007      0.000       0.544       0.579
BsmtFinType2_Unf       